## **Data Reading**

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import * 

df = spark.read.format("parquet")\
                .load("abfss://bronze@storagetosty.dfs.core.windows.net/customers")

display(df)

## **Column Transformation**